<a href="https://colab.research.google.com/github/Rishit-dagli/ConvMixer-torch2tf/blob/main/conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conversion Scripts

Authors: [Rishit Dagli](https://twitter.com/rishit_dagli)

In this Notebook we convert these 3 ConvMixer models to TensorFlow SavedModels:

| Model Name | resolution | acc@1 | #params | File Size | TensorFlow Model |
|------------|:---:|:---:|:---:|----------:|:--------------:|
| ConvMixer-1536/20 | 224x224 | 81.37 | 51.6 x 10^6 | 184MB | [github](https://github.com/Rishit-dagli/ConvMixer-torch2tf/releases/download/untagged-de81892a1b06347b8d97/convmixer_1536_20.tar.gz)/[drive](https://drive.google.com/file/d/1qrzap4vi2KFQTHxf9h_AMbWGvtbP5rIA/view?usp=sharing)/[bucket](https://storage.googleapis.com/convmixer-hubmodels.appspot.com/convmixer_1536_20.tar.gz) |
| ConvMixer-768/32 | 224x224 | 80.16 | 21.1 x 10^6 | 75MB | [github](https://github.com/Rishit-dagli/ConvMixer-torch2tf/releases/download/untagged-de81892a1b06347b8d97/convmixer_768_32.tar.gz)/[drive](https://drive.google.com/file/d/1NJgHKjPd3YC8XHypQIs5A05XKd15o0s3/view?usp=sharing)/[bucket](https://storage.googleapis.com/convmixer-hubmodels.appspot.com/convmixer_768_32.tar.gz) |
| ConvMixer-1024/20 | 224x224 | 76.94 | 24.4 x 10^6 | 87MB | [github](https://github.com/Rishit-dagli/ConvMixer-torch2tf/releases/download/untagged-de81892a1b06347b8d97/convmixer_1024_20.tar.gz)/[drive](https://drive.google.com/file/d/1--jRgK0KmLtWCJswYtfxSIfEcjAOrJyv/view?usp=sharing)/[bucket](https://storage.googleapis.com/convmixer-hubmodels.appspot.com/convmixer_1024_20.tar.gz) |

## Setup

We start off by installing [`timm`](https://github.com/rwightman/pytorch-image-models) but notice we are installing from my fork of it, this fork includes code from `timm_convmixer.py` from this repo. This updates the way the model handles padding in a `nn.Conv2D` layer to allow it to be converted.

In [ ]:
!pip install git+https://github.com/Rishit-dagli/pytorch-image-models.git@Rishit-dagli-convmixer-tf#egg=timm

  Cloning https://github.com/Rishit-dagli/pytorch-image-models.git (to revision Rishit-dagli-convmixer-tf) to /tmp/pip-install-e8uqgzff/timm_e597bf84cf2d47059e0ea3746c4ec313
  Running command git clone -q https://github.com/Rishit-dagli/pytorch-image-models.git /tmp/pip-install-e8uqgzff/timm_e597bf84cf2d47059e0ea3746c4ec313
  Running command git checkout -b Rishit-dagli-convmixer-tf --track origin/Rishit-dagli-convmixer-tf
  Switched to a new branch 'Rishit-dagli-convmixer-tf'
  Branch 'Rishit-dagli-convmixer-tf' set up to track remote branch 'Rishit-dagli-convmixer-tf' from 'origin'.
  Created wheel for timm: filename=timm-0.5.0-py3-none-any.whl size=417714 sha256=356aa1cab8ace2c32f2536b6851573a6721558de533feffb56e0630625d508b9
  Stored in directory: /tmp/pip-ephem-wheel-cache-astx_hvu/wheels/29/a9/a4/4ad1e14f936318e78dec16e4541708836546145fafa44e5915
Successfully built timm


In [ ]:
!pip install --pre torch torchvision torchaudio -f https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html --upgrade

Looking in links: https://download.pytorch.org/whl/nightly/cpu/torch_nightly.html
     |████████████████████████████████| 151.2 MB 17 kB/s 
     |████████████████████████████████| 16.8 MB 59 kB/s 
     |████████████████████████████████| 2.7 MB 44.8 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu111
    Uninstalling torch-1.9.0+cu111:
      Successfully uninstalled torch-1.9.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu111
    Uninstalling torchvision-0.10.0+cu111:
      Successfully uninstalled torchvision-0.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.11.0.dev20211023+cpu which is incompatible.


In [ ]:
!pip install onnx onnx-tf

     |████████████████████████████████| 12.3 MB 7.3 MB/s 
     |████████████████████████████████| 222 kB 64.5 MB/s 
     |████████████████████████████████| 1.1 MB 43.3 MB/s 


In [ ]:
import os
from io import BytesIO

import numpy as np
import onnx
import requests
import tensorflow as tf
import tensorflow_hub as hub
import timm
import torch
from onnx_tf.backend import prepare
from PIL import Image

## Choose pretrained model

In [ ]:
# fmt: off

#@title Choose a model type
model_variant = "convmixer_1024_20_ks9_p14" #@param ['convmixer_1536_20', 'convmixer_768_32', 'convmixer_1024_20_ks9_p14']
resolution = [224, 224] 
num_classes = 1000

# fmt: on

## Loading PyTorch pre-trained Model

We will now load the pre-trained PyTorch models.

In [ ]:
model = timm.create_model(model_variant, pretrained=True)

Downloading: "https://github.com/tmp-iclr/convmixer/releases/download/timm-v1.0/convmixer_1024_20_ks9_p14.pth.tar" to /root/.cache/torch/hub/checkpoints/convmixer_1024_20_ks9_p14.pth.tar


### Transpose channel axes

Often in PyTorch we use the channels first data format meaning you would want to pass in input $(N, C_{in}, H_{in}, W_{in})$ wwhereas in TensorFlow you would often want to use the channels last format; pass in input as $(N, H_{in}, W_{in}, C_{in})$. You can also notice that the default data format for [nn.Conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#torch.nn.Conv2d) is channels first and for [tf.keras.layers.Conv2D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) is channels last.

To do so we create a wrapper model over the earlier model we created which just transposes the channel axes in the input tensor.

In [ ]:
class TransposeModelWrapper(torch.nn.Module):
    def __init__(self, main_module):
        super(TransposeModelWrapper, self).__init__()
        self.main_module = main_module

    def forward(self, x):
        x = torch.transpose(x, 1, 3)
        return self.main_module(x)


transposed_model = TransposeModelWrapper(model)

## Convert to ONNX

We set `dynamic_axes` to allow the converted model operate with arbitrary batch sizes.

In [ ]:
batch_size = 1
x = torch.randn(batch_size, resolution[0], resolution[1], 3)
model_onnx = model_variant + ".onnx"

torch.onnx.export(
    transposed_model,
    x,
    model_onnx,
    opset_version=9,
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},
)

## Create a TensorFlow SavedModel

We will first load the onnx model and then check that the IR is well formed.

In [ ]:
onnx_model = onnx.load(model_onnx)
onnx.checker.check_model(onnx_model)

We will now use the ONNX TensorFlow backend to convert our model to a TensorFlow SavedModel.

In [ ]:
os.environ['onnx_model_dir'] = "/content/" + model_variant + ".onnx"
os.environ['saved_model_dir'] = "/content/tf_" + model_variant

In [ ]:
!onnx-tf convert -i $onnx_model_dir -o $saved_model_dir

2021-10-23 16:06:25,111 - onnx-tf - INFO - Start converting onnx pb to tf saved model
2021-10-23 16:06:25.553011: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-10-23 16:06:25.553097: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (b225fa41d07c): /proc/driver/nvidia/version does not exist
2021-10-23 16:06:38.647195: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2021-10-23 16:15:32,177 - onnx-tf - INFO - Converting completes successfully.
INFO:onnx-tf:Converting completes successfully.


## Verify SavedModel works as expected

In [ ]:
saved_model_dir = "/content/tf_" + model_variant

### Load SavedModel

In [ ]:
model = tf.saved_model.load(saved_model_dir)

In [ ]:
layer = hub.KerasLayer(
    saved_model_dir, signature="serving_default", signature_outputs_as_dict=True
)

### Check input TypeSpec

We will now use the SavedModel CLI to verify that our SavedModel has the correct input type spec which should be $(-1, 224, 224, 3)$ depending on which model you converted.

In [ ]:
!saved_model_cli show --dir $saved_model_dir --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 224, 224, 3)
      name: serving_default_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['output'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1000)
      name: PartitionedCall:0
Method name is: tensorflow/serving/predict


## Inference with the model

We will now try inferencing with our TensorFlow SavedModel first on a dummy tensor and then on an image.

### Dummy Tensor

In the first example we try on a dummy tensor with $N=5$ to verify using an arbitary batch size works well. The output of these code cells should be:

- `TensorShape([5, 1000])` or `TensorShape([5, 21841])`
- `TensorShape([1, 1000])` or `TensorShape([1, 21841])`

depending on which model you convert.

In [ ]:
dummy = model.signatures["serving_default"](
    tf.random.normal([5, resolution[0], resolution[1], 3])
)
list(dummy.values())[0].shape

TensorShape([5, 1000])

In [ ]:
dummy = layer(tf.random.normal([1, resolution[0], resolution[1], 3]))
list(dummy.values())[0].shape

TensorShape([1, 1000])

### On a real image

#### Image preprocessing utilities (adapted from [Willi Gierke](https://ch.linkedin.com/in/willi-gierke))

In [ ]:
def preprocess_image(image):
    image = np.array(image)
    image_resized = tf.image.resize(image, (resolution[0], resolution[1]))
    image_resized = tf.cast(image_resized, tf.float32)
    image_resized = image_resized / 255
    image_resized = tf.keras.layers.Normalization(
        mean=(0.485, 0.456, 0.406), variance=(0.052441, 0.050176, 0.050625)
    )(image_resized)
    return tf.expand_dims(image_resized, 0).numpy()


def load_image_from_url(url):
    response = requests.get(url)
    image = Image.open(BytesIO(response.content))
    image = preprocess_image(image)
    return image
    
!wget https://storage.googleapis.com/bit_models/ilsvrc2012_wordnet_lemmas.txt -O ilsvrc2012_wordnet_lemmas.txt

--2021-10-23 16:15:53--  https://storage.googleapis.com/bit_models/ilsvrc2012_wordnet_lemmas.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.195.128, 173.194.202.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21675 (21K) [text/plain]
Saving to: ‘ilsvrc2012_wordnet_lemmas.txt’

ilsvrc2012_wordnet_ 100%[===================>]  21.17K  --.-KB/s    in 0s      

2021-10-23 16:15:53 (73.5 MB/s) - ‘ilsvrc2012_wordnet_lemmas.txt’ saved [21675/21675]



#### Load image and infer

In [ ]:
with open("ilsvrc2012_wordnet_lemmas.txt", "r") as f:
    lines = f.readlines()
imagenet_int_to_str = [line.rstrip() for line in lines]

def infer_on_image(img_url, expected_label):
    image = load_image_from_url(img_url)
    predictions = model.signatures["serving_default"](tf.constant(image))
    logits = predictions["output"][0]
    predicted_label = imagenet_int_to_str[int(np.argmax(logits))]
    assert (
        predicted_label == expected_label
    ), f"Expected {expected_label} but was {predicted_label}"

In [ ]:
infer_on_image(img_url = "https://i.imgur.com/IKR2LyP.jpg", expected_label = "purse")

As a sanity test lets try 5 more images.

In [ ]:
infer_on_image(img_url = "https://i.imgur.com/mH0Wrvb.jpg", expected_label = "goldfish, Carassius_auratus")
infer_on_image(img_url = "https://i.imgur.com/A5m4ZG1.jpg", expected_label = "scorpion")
infer_on_image(img_url = "https://i.imgur.com/faOAEFg.jpg", expected_label = "leatherback_turtle, leatherback, leathery_turtle, Dermochelys_coriacea")
infer_on_image(img_url = "https://i.imgur.com/lfhdaSi.jpg", expected_label = "Siamese_cat, Siamese")
infer_on_image(img_url = "https://i.imgur.com/Qwa8wHX.jpg", expected_label = "boa_constrictor, Constrictor_constrictor")

## References

[1] Anonymous. Patches Are All You Need? 2021. openreview.net, https://openreview.net/forum?id=TVHS5Y4dNvM.

[2] Official Code Implementation: https://github.com/tmp-iclr/convmixer

[3] Ross Wightman, . "PyTorch Image Models." https://github.com/rwightman/pytorch-image-models.